In [2]:
# 1. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 2. Import necessary libraries [cite: 39]
import pandas as pd
import numpy as np
import os

# 3. Define file paths
# UPDATE THIS PATH to match where you uploaded the files in your Drive
base_path = '/content/drive/MyDrive/Retail_Project/data/'

# 4. Load the data
try:
    train  = pd.read_csv(os.path.join(base_path, 'train.csv'), low_memory=False)
    store  = pd.read_csv(os.path.join(base_path, 'store.csv'))
    test  = pd.read_csv(os.path.join(base_path, 'test.csv'))


    print("✅ Data loaded successfully!")
    print(f"Sales Data Shape: {df_sales.shape}")
    print(f"Store Data Shape: {df_store.shape}")

    # Display first few rows to verify
    display(df_sales.head())
    display(df_store.head())

except FileNotFoundError:
    print("❌ Error: Files not found. Please check your base_path variable.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Data loaded successfully!
Sales Data Shape: (1017209, 9)
Store Data Shape: (1115, 10)


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [3]:
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [4]:
store.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [5]:
test.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0


In [6]:
import pandas as pd
import numpy as np

# --- Merge ---
train_m = train.merge(store, on="Store", how="left")
test_m  = test.merge(store,  on="Store", how="left")

# --- Standardize types ---
def standardize(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # Ensure these are strings (categorical)
    if "StateHoliday" in df.columns:
        df["StateHoliday"] = df["StateHoliday"].astype(str)
        df["StateHoliday"] = df["StateHoliday"].replace({"0": "0", "0.0": "0"})

    for col in ["StoreType", "Assortment", "PromoInterval"]:
        if col in df.columns:
            df[col] = df[col].astype("string")

    # Ensure numeric flags are int
    for col in ["Promo", "SchoolHoliday", "Open", "Promo2"]:
        if col in df.columns:
            df[col] = df[col].fillna(0).astype(int)

    return df

train_m = standardize(train_m)
test_m  = standardize(test_m)

print("Merged train:", train_m.shape, "Merged test:", test_m.shape)


Merged train: (1017209, 18) Merged test: (41088, 17)


In [7]:
def fill_missing(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # Numeric columns (fill with median)
    num_cols = [
        "CompetitionDistance",
        "CompetitionOpenSinceMonth", "CompetitionOpenSinceYear",
        "Promo2SinceWeek", "Promo2SinceYear"
    ]
    for c in num_cols:
        if c in df.columns:
            if df[c].isna().any():
                med = df[c].median()
                df[c] = df[c].fillna(med)

    # Categorical columns (fill with "Unknown"/None)
    cat_cols = ["StoreType", "Assortment", "PromoInterval"]
    for c in cat_cols:
        if c in df.columns:
            df[c] = df[c].fillna("None")

    # StateHoliday (fill with "0")
    if "StateHoliday" in df.columns:
        df["StateHoliday"] = df["StateHoliday"].fillna("0")

    return df

train_m = fill_missing(train_m)
test_m  = fill_missing(test_m)

print("Missing values (train) top 10:")
print(train_m.isna().sum().sort_values(ascending=False).head(10))


Missing values (train) top 10:
Store            0
DayOfWeek        0
Date             0
Sales            0
Customers        0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
StoreType        0
dtype: int64


In [8]:
train_m = train_m.sort_values(["Store", "Date"]).reset_index(drop=True)
test_m  = test_m.sort_values(["Store", "Date"]).reset_index(drop=True)

before = train_m.shape[0]
train_m_clean = train_m[train_m["Open"] == 1].copy()
after = train_m_clean.shape[0]

print("Train rows before:", before)
print("Train rows after Open==1 filter:", after)
print("Dropped rows:", before - after)

# Quick sanity
print("Sales min/max:", train_m_clean["Sales"].min(), train_m_clean["Sales"].max())


Train rows before: 1017209
Train rows after Open==1 filter: 844392
Dropped rows: 172817
Sales min/max: 0 41551


In [10]:
import os

os.makedirs("/content/drive/MyDrive/Retail_Project/data/", exist_ok=True)

train_path = "/content/drive/MyDrive/Retail_Project/data/train_merged_clean.parquet"
test_path  = "/content/drive/MyDrive/Retail_Project/data/test_merged.parquet"

train_m_clean.to_parquet(train_path, index=False)
test_m.to_parquet(test_path, index=False)

print("Saved:", train_path)
print("Saved:", test_path)

print("Reload check:")
tmp = pd.read_parquet(train_path)
print(tmp.shape, tmp.columns[:10].tolist())


Saved: /content/drive/MyDrive/Retail_Project/data/train_merged_clean.parquet
Saved: /content/drive/MyDrive/Retail_Project/data/test_merged.parquet
Reload check:
(844392, 18) ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday', 'StoreType']


In [11]:
import pandas as pd
import numpy as np

train_path = "/content/drive/MyDrive/Retail_Project/data/train_merged_clean.parquet"
df = pd.read_parquet(train_path)

df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values(["Store", "Date"]).reset_index(drop=True)

max_date = df["Date"].max()
cutoff_date = max_date - pd.Timedelta(days=42)  # last 6 weeks validation

train_df = df[df["Date"] <= cutoff_date].copy()
valid_df = df[df["Date"] > cutoff_date].copy()

print("Max date:", max_date)
print("Cutoff :", cutoff_date)
print("Train :", train_df.shape, "Valid:", valid_df.shape)
print("Valid date range:", valid_df["Date"].min(), "→", valid_df["Date"].max())


Max date: 2015-07-31 00:00:00
Cutoff : 2015-06-19 00:00:00
Train : (804110, 18) Valid: (40282, 18)
Valid date range: 2015-06-20 00:00:00 → 2015-07-31 00:00:00


In [12]:
# Mean sales per (Store, DayOfWeek) from TRAIN only
grp = train_df.groupby(["Store", "DayOfWeek"])["Sales"].mean().reset_index()
grp = grp.rename(columns={"Sales": "pred_dow_mean"})

# Store fallback mean (if needed)
store_mean = train_df.groupby("Store")["Sales"].mean().reset_index()
store_mean = store_mean.rename(columns={"Sales": "pred_store_mean"})


In [13]:
pred1 = valid_df.merge(grp, on=["Store", "DayOfWeek"], how="left")
pred1 = pred1.merge(store_mean, on="Store", how="left")

# If missing (rare), fallback to store mean, else overall mean
overall_mean = train_df["Sales"].mean()
pred1["yhat_dow_mean"] = pred1["pred_dow_mean"].fillna(pred1["pred_store_mean"]).fillna(overall_mean)

y_true = pred1["Sales"].values
y_hat1 = pred1["yhat_dow_mean"].values

print("Baseline-1 ready:", len(y_hat1))


Baseline-1 ready: 40282


In [15]:
# Build a lookup table from FULL df (train+valid) so we can fetch t-7 sales
lookup = df[["Store", "Date", "Sales"]].copy()
# Shift the Date column to represent the target date for which sales_t_minus_7 is a feature
lookup["Date"] = lookup["Date"] + pd.Timedelta(days=7)
lookup = lookup.rename(columns={"Sales": "sales_t_minus_7"})

pred2 = valid_df.merge(lookup, on=["Store", "Date"], how="left")

# Fallback to Baseline-1 when lag7 is missing
pred2 = pred2.merge(grp, on=["Store", "DayOfWeek"], how="left")
pred2 = pred2.merge(store_mean, on="Store", how="left")

fallback = pred2["pred_dow_mean"].fillna(pred2["pred_store_mean"]).fillna(overall_mean)
pred2["yhat_lag7"] = pred2["sales_t_minus_7"].fillna(fallback)

y_hat2 = pred2["yhat_lag7"].values

print("Baseline-2 ready:", len(y_hat2))
print("Lag7 missing count:", pred2["sales_t_minus_7"].isna().sum())

Baseline-2 ready: 40282
Lag7 missing count: 13


In [16]:
import numpy as np

def mae(y, yhat):
    return float(np.mean(np.abs(y - yhat)))

def rmse(y, yhat):
    return float(np.sqrt(np.mean((y - yhat) ** 2)))

def mape(y, yhat):
    y = np.array(y, dtype=float)
    yhat = np.array(yhat, dtype=float)
    denom = np.maximum(np.abs(y), 1e-9)
    return float(np.mean(np.abs((y - yhat) / denom)) * 100)

results = pd.DataFrame({
    "Model": ["Baseline-1: Store+DoW mean", "Baseline-2: Lag-7 days"],
    "MAE":  [mae(y_true, y_hat1), mae(y_true, y_hat2)],
    "RMSE": [rmse(y_true, y_hat1), rmse(y_true, y_hat2)],
    "MAPE%":[mape(y_true, y_hat1), mape(y_true, y_hat2)]
})

results


,Model,MAE,RMSE,MAPE%
0,Baseline-1: Store+DoW mean,1254.176000,1664.700963,18.511693
1,Baseline-2: Lag-7 days,2189.584704,2821.639452,32.077273


In [17]:
!pip -q install lightgbm


In [21]:
import pandas as pd
import numpy as np

df = pd.read_parquet("/content/drive/MyDrive/Retail_Project/data/train_merged_clean.parquet")
df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values(["Store", "Date"]).reset_index(drop=True)

# Extra time features
df["Year"]  = df["Date"].dt.year
df["Month"] = df["Date"].dt.month
df["Day"]   = df["Date"].dt.day
df["Week"]  = df["Date"].dt.isocalendar().week.astype(int)
df["IsWeekend"] = (df["DayOfWeek"].isin([6, 7])).astype(int)

# Competition months open
comp_start = pd.to_datetime(
    dict(year=df["CompetitionOpenSinceYear"].astype(int),
         month=df["CompetitionOpenSinceMonth"].astype(int),
         day=1),
    errors="coerce"
)
df["CompMonthsOpen"] = (
    (df["Date"].dt.year - comp_start.dt.year) * 12 +
    (df["Date"].dt.month - comp_start.dt.month)
).fillna(0).clip(lower=0)

# Promo2 start date (ISO year-week -> Monday)
promo2_start = pd.to_datetime(
    df["Promo2SinceYear"].astype(int).astype(str) + "-W" +
    df["Promo2SinceWeek"].astype(int).astype(str) + "-1",
    format="%G-W%V-%u",
    errors="coerce"
)
df["Promo2WeeksActive"] = ((df["Date"] - promo2_start).dt.days / 7.0).fillna(0).clip(lower=0)

# Promo2 month active (uses PromoInterval months)
month_str = df["Date"].dt.strftime("%b")  # Jan, Feb, ...
def promo2_month_active(row):
    if row["Promo2"] != 1:
        return 0
    pi = row["PromoInterval"]
    if pd.isna(pi) or pi in ["None", "nan"]:
        return 0
    months = [m.strip() for m in str(pi).split(",")]
    return int(month_str.loc[row.name] in months)

df["Promo2MonthActive"] = df.apply(promo2_month_active, axis=1)

# Categorical cleanup
df["StateHoliday"] = df["StateHoliday"].astype(str).replace({"0.0":"0"})
for c in ["StoreType", "Assortment", "PromoInterval", "StateHoliday"]:
    df[c] = df[c].astype("category")

df.shape


(844392, 26)

In [22]:
g = df.groupby("Store")["Sales"]

df["sales_lag_1"]  = g.shift(1)
df["sales_lag_7"]  = g.shift(7)
df["sales_lag_14"] = g.shift(14)

df["sales_rm_7"]  = g.transform(lambda s: s.shift(1).rolling(7).mean())
df["sales_rm_14"] = g.transform(lambda s: s.shift(1).rolling(14).mean())
df["sales_rm_28"] = g.transform(lambda s: s.shift(1).rolling(28).mean())

df["sales_rs_7"]  = g.transform(lambda s: s.shift(1).rolling(7).std())
df["sales_rs_14"] = g.transform(lambda s: s.shift(1).rolling(14).std())

# Drop rows where lags aren't available (early days per store)
before = len(df)
df = df.dropna(subset=["sales_lag_1","sales_lag_7","sales_rm_7","sales_rm_14","sales_rm_28"]).copy()
after = len(df)

print("Rows before:", before, "after lag-drop:", after, "dropped:", before-after)
df.head(2)


Rows before: 844392 after lag-drop: 813172 dropped: 31220


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,...,Promo2WeeksActive,Promo2MonthActive,sales_lag_1,sales_lag_7,sales_lag_14,sales_rm_7,sales_rm_14,sales_rm_28,sales_rs_7,sales_rs_14
28,1,1,2013-02-04,7032,762,1,1,0,0,c,...,36.000000,0,5970.0,5598.0,4127.0,4898.714286,5076.642857,5001.214286,855.366727,710.572966
29,1,2,2013-02-05,6049,678,1,1,0,0,c,...,36.142857,0,7032.0,4055.0,5182.0,5103.571429,5284.142857,5054.857143,1166.051436,826.610024


In [23]:
from lightgbm import LGBMRegressor

max_date = df["Date"].max()
cutoff_date = max_date - pd.Timedelta(days=42)

train_df = df[df["Date"] <= cutoff_date].copy()
valid_df = df[df["Date"] > cutoff_date].copy()

target = "Sales"

# IMPORTANT: Customers is not in test dataset -> don't use it
drop_cols = ["Sales", "Customers", "Date"]  # Date not directly used
feature_cols = [c for c in df.columns if c not in drop_cols]

X_train, y_train = train_df[feature_cols], np.log1p(train_df[target].values)
X_valid, y_valid = valid_df[feature_cols], np.log1p(valid_df[target].values)

cat_cols = ["StoreType", "Assortment", "PromoInterval", "StateHoliday"]
for c in cat_cols:
    if c in X_train.columns:
        X_train[c] = X_train[c].astype("category")
        X_valid[c] = X_valid[c].astype("category")

model = LGBMRegressor(
    n_estimators=5000,
    learning_rate=0.03,
    num_leaves=64,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

model.fit(
    X_train, y_train,
    eval_set=[(X_valid, y_valid)],
    eval_metric="rmse",
    categorical_feature=cat_cols,
    callbacks=[],
)

# Predict back to original scale
pred_log = model.predict(X_valid)
y_hat = np.expm1(pred_log)
y_true = valid_df["Sales"].values

def mae(y, yhat):  return float(np.mean(np.abs(y - yhat)))
def rmse(y, yhat): return float(np.sqrt(np.mean((y - yhat) ** 2)))
def mape(y, yhat):
    denom = np.maximum(np.abs(y), 1e-9)
    return float(np.mean(np.abs((y - yhat) / denom)) * 100)

print("LightGBM  MAE :", mae(y_true, y_hat))
print("LightGBM  RMSE:", rmse(y_true, y_hat))
print("LightGBM  MAPE:", mape(y_true, y_hat))

/tmp/ipython-input-901171778.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[c] = X_train[c].astype("category")
/tmp/ipython-input-901171778.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_valid[c] = X_valid[c].astype("category")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.466042 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3267
[LightGBM] [Info] Number of data points in the train set: 772890, number of used features: 30
[LightGBM] [Info] Start training from score 8.761272
LightGBM  MAE : 535.9093162740279
LightGBM  RMSE: 770.4584122543961
LightGBM  MAPE: 7.9876160637426725


In [24]:
import os, json
import joblib

ART_DIR = "/content/drive/MyDrive/Retail_Project/data/"
os.makedirs(ART_DIR, exist_ok=True)

meta = {
    "feature_cols": feature_cols,
    "cat_cols": cat_cols,
    "cutoff_days_valid": 42,
    "train_max_date": str(df["Date"].max().date()),
}

# Save category levels so inference uses same mappings
cat_levels = {}
for c in cat_cols:
    if c in df.columns and str(df[c].dtype) == "category":
        cat_levels[c] = [str(x) for x in df[c].cat.categories.tolist()]
meta["cat_levels"] = cat_levels

with open(f"{ART_DIR}/meta.json", "w") as f:
    json.dump(meta, f, indent=2)

print("Saved meta.json with", len(feature_cols), "features and", len(cat_levels), "cat columns")


Saved meta.json with 31 features and 4 cat columns


In [25]:
joblib.dump(model, f"{ART_DIR}/lgbm_model.joblib")
print("Saved model to:", f"{ART_DIR}/lgbm_model.joblib")
!ls -lh {ART_DIR}


Saved model to: /content/drive/MyDrive/Retail_Project/data//lgbm_model.joblib
total 140M
-rw------- 1 root root  63M Dec 11 16:02 df_cleaned.csv
-rw------- 1 root root  29M Dec 12 18:12 lgbm_model.joblib
-rw------- 1 root root 1.1K Dec 12 18:11 meta.json
-rw------- 1 root root 7.0M Dec 11 15:35 rossmann-store-sales.zip
-rw------- 1 root root 311K Dec 11 15:37 sample_submission.csv
-rw------- 1 root root  44K Dec 11 15:37 store.csv
-rw------- 1 root root 1.4M Dec 11 15:37 test.csv
-rw------- 1 root root 275K Dec 12 17:30 test_merged.parquet
-rw------- 1 root root  37M Dec 11 15:37 train.csv
-rw------- 1 root root 3.2M Dec 12 17:30 train_merged_clean.parquet


In [26]:
import pandas as pd
import numpy as np
import json, joblib

def add_exogenous_features(df_in: pd.DataFrame) -> pd.DataFrame:
    df = df_in.copy()
    df["Date"] = pd.to_datetime(df["Date"])
    df = df.sort_values(["Store", "Date"]).reset_index(drop=True)

    # Time features
    df["Year"]  = df["Date"].dt.year
    df["Month"] = df["Date"].dt.month
    df["Day"]   = df["Date"].dt.day
    df["Week"]  = df["Date"].dt.isocalendar().week.astype(int)
    df["IsWeekend"] = (df["DayOfWeek"].isin([6, 7])).astype(int)

    # Competition months open
    comp_start = pd.to_datetime(
        dict(year=df["CompetitionOpenSinceYear"].astype(int),
             month=df["CompetitionOpenSinceMonth"].astype(int),
             day=1),
        errors="coerce"
    )
    df["CompMonthsOpen"] = (
        (df["Date"].dt.year - comp_start.dt.year) * 12 +
        (df["Date"].dt.month - comp_start.dt.month)
    ).fillna(0).clip(lower=0)

    # Promo2 start date
    promo2_start = pd.to_datetime(
        df["Promo2SinceYear"].astype(int).astype(str) + "-W" +
        df["Promo2SinceWeek"].astype(int).astype(str) + "-1",
        format="%G-W%V-%u",
        errors="coerce"
    )
    df["Promo2WeeksActive"] = ((df["Date"] - promo2_start).dt.days / 7.0).fillna(0).clip(lower=0)

    # Promo2 month active
    month_str = df["Date"].dt.strftime("%b")
    def promo2_month_active(i):
        if int(df.loc[i, "Promo2"]) != 1:
            return 0
        pi = df.loc[i, "PromoInterval"]
        if pd.isna(pi) or str(pi) in ["None", "nan"]:
            return 0
        months = [m.strip() for m in str(pi).split(",")]
        return int(month_str.loc[i] in months)

    df["Promo2MonthActive"] = [promo2_month_active(i) for i in range(len(df))]

    return df


In [28]:
import json, joblib
import pandas as pd
import numpy as np

ART_DIR = "/content/drive/MyDrive/Retail_Project/data/"

with open(f"{ART_DIR}/meta.json", "r") as f:
    meta = json.load(f)

loaded_model = joblib.load(f"{ART_DIR}/lgbm_model.joblib")
feature_cols_loaded = meta["feature_cols"]
cat_cols_loaded = meta["cat_cols"]
cat_levels = meta["cat_levels"]

# Take a small slice from valid_df (we already have it from Step 4)
sample = valid_df.head(200).copy()

# Rebuild only exogenous features (lags already exist in valid_df)
sample = add_exogenous_features(sample)

# Enforce categorical levels
for c in cat_cols_loaded:
    if c in sample.columns:
        sample[c] = sample[c].astype(str).fillna("None")
        if c in cat_levels:
            sample[c] = pd.Categorical(sample[c], categories=cat_levels[c])

X = sample[feature_cols_loaded].copy()
pred = np.expm1(loaded_model.predict(X))

print("Pred OK. preds:", pred[:5])
print("Feature match:", len(feature_cols_loaded), "cols")


Pred OK. preds: [4086.58865368 3702.01614133 3562.93766489 3457.66119319 3303.4772036 ]
Feature match: 31 cols


In [29]:
import pandas as pd
import numpy as np
import json, joblib
from pathlib import Path

ART_DIR = "/content/drive/MyDrive/Retail_Project/data/"

with open(f"{ART_DIR}/meta.json", "r") as f:
    meta = json.load(f)

model = joblib.load(f"{ART_DIR}/lgbm_model.joblib")
feature_cols = meta["feature_cols"]
cat_cols = meta["cat_cols"]
cat_levels = meta["cat_levels"]

train_df = pd.read_parquet("/content/drive/MyDrive/Retail_Project/data/train_merged_clean.parquet")
test_df  = pd.read_parquet("/content/drive/MyDrive/Retail_Project/data/test_merged.parquet")

train_df["Date"] = pd.to_datetime(train_df["Date"])
test_df["Date"]  = pd.to_datetime(test_df["Date"])

# Open can be missing in test -> treat as open (1)
if "Open" in test_df.columns:
    test_df["Open"] = test_df["Open"].fillna(1).astype(int)

train_df = train_df.sort_values(["Store","Date"]).reset_index(drop=True)
test_df  = test_df.sort_values(["Store","Date"]).reset_index(drop=True)

print(train_df.shape, test_df.shape, train_df["Date"].min(), train_df["Date"].max())


(844392, 18) (41088, 17) 2013-01-01 00:00:00 2015-07-31 00:00:00


In [30]:
train_exog = add_exogenous_features(train_df)
test_exog  = add_exogenous_features(test_df)

print("exog ready:", train_exog.shape, test_exog.shape)


exog ready: (844392, 26) (41088, 25)


In [31]:
def enforce_cats(df_one: pd.DataFrame) -> pd.DataFrame:
    df_one = df_one.copy()
    for c in cat_cols:
        if c in df_one.columns:
            df_one[c] = df_one[c].astype(str).fillna("None")
            if c in cat_levels:
                df_one[c] = pd.Categorical(df_one[c], categories=cat_levels[c])
    return df_one

def recursive_predict(future_df: pd.DataFrame, history_sales: dict, store_mean: dict, overall_mean: float) -> np.ndarray:
    """
    future_df: rows for future dates with all exogenous/store columns present (no Sales needed)
    history_sales: store -> list of past open-day sales (actuals initially, then we append predictions)
    """
    preds = np.zeros(len(future_df), dtype=float)

    # group indices by store for speed
    store_to_idx = future_df.groupby("Store").indices

    for store, idx in store_to_idx.items():
        idx = list(idx)
        idx.sort(key=lambda i: future_df.loc[i, "Date"])

        hist = history_sales.get(store, [])
        fallback_mean = store_mean.get(store, overall_mean)

        for i in idx:
            row = future_df.iloc[i]

            # If store closed, predict 0 and DON'T update open-day history
            if "Open" in future_df.columns and int(row["Open"]) == 0:
                preds[i] = 0.0
                continue

            # safety fallback if somehow history is short
            def get_lag(k):
                return hist[-k] if len(hist) >= k else fallback_mean

            def roll_mean(w):
                return float(np.mean(hist[-w:])) if len(hist) >= w else float(np.mean(hist)) if len(hist) else fallback_mean

            def roll_std(w):
                return float(np.std(hist[-w:], ddof=0)) if len(hist) >= w else float(np.std(hist, ddof=0)) if len(hist) else 0.0

            row_dict = row.to_dict()

            # lag/rolling features (same names as training)
            row_dict["sales_lag_1"]  = get_lag(1)
            row_dict["sales_lag_7"]  = get_lag(7)
            row_dict["sales_lag_14"] = get_lag(14)

            row_dict["sales_rm_7"]  = roll_mean(7)
            row_dict["sales_rm_14"] = roll_mean(14)
            row_dict["sales_rm_28"] = roll_mean(28)

            row_dict["sales_rs_7"]  = roll_std(7)
            row_dict["sales_rs_14"] = roll_std(14)

            df_one = pd.DataFrame([row_dict])

            # enforce categorical levels
            df_one = enforce_cats(df_one)

            # align to expected features
            X = df_one.reindex(columns=feature_cols)

            pred_log = model.predict(X)[0]
            pred = float(np.expm1(pred_log))
            pred = max(pred, 0.0)

            preds[i] = pred
            hist.append(pred)

        history_sales[store] = hist  # store back

    return preds


In [32]:
# backtest split
max_date = train_exog["Date"].max()
cutoff_date = max_date - pd.Timedelta(days=42)

train_part = train_exog[train_exog["Date"] <= cutoff_date].copy()
valid_part = train_exog[train_exog["Date"] > cutoff_date].copy()

# seed history with TRAIN PART actual sales
history = {s: grp["Sales"].tolist() for s, grp in train_part.groupby("Store")}
store_mean = train_part.groupby("Store")["Sales"].mean().to_dict()
overall_mean = float(train_part["Sales"].mean())

# predict recursively on valid
valid_future = valid_part.drop(columns=["Sales","Customers"], errors="ignore").copy()
pred_valid = recursive_predict(valid_future.reset_index(drop=True), history, store_mean, overall_mean)

y_true = valid_part["Sales"].values

def mae(y, yhat):  return float(np.mean(np.abs(y - yhat)))
def rmse(y, yhat): return float(np.sqrt(np.mean((y - yhat) ** 2)))
def mape(y, yhat):
    denom = np.maximum(np.abs(y), 1e-9)
    return float(np.mean(np.abs((y - yhat) / denom)) * 100)

print("Recursive Backtest MAE :", mae(y_true, pred_valid))
print("Recursive Backtest RMSE:", rmse(y_true, pred_valid))
print("Recursive Backtest MAPE:", mape(y_true, pred_valid))


Recursive Backtest MAE : 604.8186704840775
Recursive Backtest RMSE: 883.8782507257259
Recursive Backtest MAPE: 8.92088689439205


In [33]:
# seed history with FULL TRAIN actual sales
history_full = {s: grp["Sales"].tolist() for s, grp in train_exog.groupby("Store")}
store_mean_full = train_exog.groupby("Store")["Sales"].mean().to_dict()
overall_mean_full = float(train_exog["Sales"].mean())

test_future = test_exog.copy()  # has Id
pred_test = recursive_predict(test_future.reset_index(drop=True), history_full, store_mean_full, overall_mean_full)

sub = pd.DataFrame({"Id": test_exog["Id"].values, "Sales": pred_test})
sub_path = f"{ART_DIR}/submission.csv"
sub.to_csv(sub_path, index=False)

print("Saved:", sub_path)
print(sub.head())


Saved: /content/drive/MyDrive/Retail_Project/data//submission.csv
      Id        Sales
0  40233  4947.596621
1  39377     0.000000
2  38521  6010.089088
3  37665  5119.911966
4  36809  4748.248554


In [34]:
!pip -q install scipy


In [37]:
import pandas as pd
import numpy as np
import json, joblib
from scipy.stats import norm

ART_DIR = "/content/drive/MyDrive/Retail_Project/data/"

with open(f"{ART_DIR}/meta.json", "r") as f:
    meta = json.load(f)

model = joblib.load(f"{ART_DIR}/lgbm_model.joblib")
feature_cols = meta["feature_cols"]
cat_cols = meta["cat_cols"]
cat_levels = meta["cat_levels"]

# Load the same lagged dataset you trained on (must include lag/rolling cols)
df = pd.read_parquet("/content/drive/MyDrive/Retail_Project/data/train_merged_clean.parquet")
df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values(["Store","Date"]).reset_index(drop=True)

# Recreate exogenous features (use your existing function)
df = add_exogenous_features(df)

# Recreate lag/rolling features (same as Step 4)
g = df.groupby("Store")["Sales"]
df["sales_lag_1"]  = g.shift(1)
df["sales_lag_7"]  = g.shift(7)
df["sales_lag_14"] = g.shift(14)
df["sales_rm_7"]   = g.transform(lambda s: s.shift(1).rolling(7).mean())
df["sales_rm_14"]  = g.transform(lambda s: s.shift(1).rolling(14).mean())
df["sales_rm_28"]  = g.transform(lambda s: s.shift(1).rolling(28).mean())
df["sales_rs_7"]   = g.transform(lambda s: s.shift(1).rolling(7).std())
df["sales_rs_14"]  = g.transform(lambda s: s.shift(1).rolling(14).std())

df = df.dropna(subset=["sales_lag_1","sales_lag_7","sales_rm_7","sales_rm_14","sales_rm_28"]).copy()

# Time split
max_date = df["Date"].max()
cutoff = max_date - pd.Timedelta(days=42)

valid = df[df["Date"] > cutoff].copy()

# enforce categorical levels
for c in cat_cols:
    if c in valid.columns:
        valid[c] = valid[c].astype(str).fillna("None")
        if c in cat_levels:
            valid[c] = pd.Categorical(valid[c], categories=cat_levels[c])

Xv = valid[feature_cols]
y_true = valid["Sales"].values
y_hat = np.expm1(model.predict(Xv))

valid["residual"] = (y_true - y_hat)

# Per-store daily sigma (fallback to global if store has too few points)
sigma_store = valid.groupby("Store")["residual"].std().to_dict()
sigma_global = float(valid["residual"].std())

print("sigma_global:", sigma_global)
print("sigma_store sample:", list(sigma_store.items())[:3])


sigma_global: 767.3311095656637
sigma_store sample: [(1, 337.7870855236249), (2, 509.4165617474778), (3, 754.5026481742684)]


In [38]:
from dataclasses import dataclass

@dataclass
class InventoryDecision:
    safety_stock: float
    reorder_point: float
    order_up_to: float
    order_qty: float

def inventory_decision(
    forecast_series: np.ndarray,  # daily forecasts for next (L+R) days
    sigma_daily: float,
    lead_time_days: int = 7,
    review_days: int = 7,
    service_level: float = 0.95,
    on_hand: float = 0.0,
    on_order: float = 0.0
) -> InventoryDecision:
    L = int(lead_time_days)
    R = int(review_days)
    assert L >= 1 and R >= 0

    z = float(norm.ppf(service_level))

    # Demand over lead time (for ROP)
    demand_L = float(np.sum(forecast_series[:L]))

    # Demand over protection period (for order-up-to)
    PP = L + R
    demand_PP = float(np.sum(forecast_series[:PP]))

    sigma_L  = (L ** 0.5) * sigma_daily
    sigma_PP = (PP ** 0.5) * sigma_daily

    ss_L  = z * sigma_L
    ss_PP = z * sigma_PP

    rop = demand_L + ss_L
    S   = demand_PP + ss_PP

    qty = max(0.0, S - (on_hand + on_order))

    return InventoryDecision(
        safety_stock=ss_PP,
        reorder_point=rop,
        order_up_to=S,
        order_qty=qty
    )


In [39]:
# Load test and predicted sales (from Step 6 artifacts)
test_exog = pd.read_parquet("/content/drive/MyDrive/Retail_Project/data/test_merged.parquet")
test_exog["Date"] = pd.to_datetime(test_exog["Date"])
test_exog = add_exogenous_features(test_exog)

# Load your submission predictions
sub = pd.read_csv(f"{ART_DIR}/submission.csv")
test_exog = test_exog.merge(sub, on="Id", how="left")  # adds predicted Sales column
test_exog = test_exog.sort_values(["Store","Date"]).reset_index(drop=True)

# Inventory parameters (defaults for demo; these will become Streamlit inputs later)
L = 7
R = 7
service = 0.95

rows = []
for store, g in test_exog.groupby("Store"):
    g = g.sort_values("Date")
    preds = g["Sales"].values  # forecasted daily demand for future dates

    sigma_d = sigma_store.get(store, sigma_global)

    # demo assumption (UI will take real values):
    on_hand = 0.0
    on_order = 0.0

    dec = inventory_decision(
        forecast_series=preds,
        sigma_daily=sigma_d,
        lead_time_days=L,
        review_days=R,
        service_level=service,
        on_hand=on_hand,
        on_order=on_order
    )

    rows.append({
        "Store": store,
        "sigma_daily": sigma_d,
        "LeadTimeDays": L,
        "ReviewDays": R,
        "ServiceLevel": service,
        "ROP": dec.reorder_point,
        "SafetyStock": dec.safety_stock,
        "OrderUpTo": dec.order_up_to,
        "RecommendedOrderQty": dec.order_qty
    })

inv_table = pd.DataFrame(rows).sort_values("RecommendedOrderQty", ascending=False)
inv_table.head(10)


,Store,sigma_daily,LeadTimeDays,ReviewDays,ServiceLevel,ROP,SafetyStock,OrderUpTo,RecommendedOrderQty
201,262,1170.621175,7,7,0.95,154682.921041,7204.563117,291558.752511,291558.752511
854,1114,1993.362623,7,7,0.95,148561.814484,12268.107854,269753.295408,269753.295408
646,842,2512.318474,7,7,0.95,138608.498852,15462.010594,256658.040813,256658.040813
433,562,1019.723965,7,7,0.95,138542.603504,6275.869447,254461.120361,254461.120361
193,251,1666.284553,7,7,0.95,123326.366207,10255.112827,221592.075956,221592.075956
293,383,1026.706987,7,7,0.95,119220.815951,6318.846304,214991.764775,214991.764775
584,756,1793.266793,7,7,0.95,119678.459858,11036.622322,214986.715333,214986.715333
567,733,698.234561,7,7,0.95,108931.147096,4297.269751,208840.025154,208840.025154
702,909,8331.325537,7,7,0.95,130810.604014,51274.965755,207224.300601,207224.300601
246,320,1157.413922,7,7,0.95,110841.279649,7123.279359,198319.795745,198319.795745


In [40]:
import json, os

ART_DIR = "/content/drive/MyDrive/Retail_Project/data/"
os.makedirs(ART_DIR, exist_ok=True)

sigma_payload = {
    "sigma_global": sigma_global,
    "sigma_store": {str(k): float(v) for k, v in sigma_store.items()}  # json-safe
}

with open(f"{ART_DIR}/sigma.json", "w") as f:
    json.dump(sigma_payload, f, indent=2)

print("Saved:", f"{ART_DIR}/sigma.json")


Saved: /content/drive/MyDrive/Retail_Project/data//sigma.json


In [41]:
!zip -r /content/rossmann_artifacts.zip /content/artifacts/rossmann_v1 > /dev/null
!zip -r /content/rossmann_processed_data.zip /content/data/processed/rossmann > /dev/null
!ls -lh /content/rossmann_*.zip


-rw-r--r-- 1 root root 3.0M Dec 12 19:18 /content/rossmann_processed_data.zip


In [48]:
from google.colab import files
import os

# Define ART_DIR as it was defined previously to locate the artifacts
ART_DIR = "/content/drive/MyDrive/Retail_Project/data/"

# Re-create rossmann_artifacts.zip from the correct directory before downloading
# This command will create a zip file named rossmann_artifacts.zip in /content/
# containing the contents of ART_DIR.
!zip -r /content/rossmann_artifacts.zip "{ART_DIR}" > /dev/null

# Now proceed with downloading both zip files
files.download("/content/rossmann_artifacts.zip")
files.download("/content/rossmann_processed_data.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [45]:
!ls

data  drive  rossmann_processed_data.zip  sample_data
